##Registros Clave-Valor (K,V)   
La clave es de tipo imnutable, cada elemento es una tupla() de dos elementos entre parentesis y separados por comas, el primer elemento es la clave y el segundo el valor.  

- **mapValues(f)** Realiza operaciones sobre los valores de los pares (K,V)  
- **reduceByKey(f)** Al llamarlo sobre un RDD de pares clave-valor (K,V), devuelve otro de pares (K,V) donde los valores de cada clave se han agregado usando la funcion dada
- **groupByKey(f)** Al llamarlo sobre un RDD de pares clave-valor (K,V), devuelve otro de pares (K,seq[V], donde los valores de cada clave se han convertido en una secuencia)
- **sortByKey()** Ordena un RDD de pares clave-valor (K,V) por clave
- **join(rdd)** Hace un Join de dos RDD de pares (K,V1)y(K,V2) y devuelve otro RDD con claves (K,(V1,V2))

DIAPOSITIVAS  
https://docs.google.com/presentation/d/1UU5Rdy3N3v0msdpE4gB8iju25vENvHNE/edit?usp=share_link&ouid=108221443273434880064&rtpof=true&sd=true

In [0]:
#creación de RDD clave-valor
palabras = sc.parallelize (['HOLA','Que', 'Tal', 'Bien', 'supercalifragilisticoespialidoso'])
pal_long = palabras.map(lambda elem: (elem, len(elem)))
pal_long.collect()

Out[1]: [('HOLA', 4),
 ('Que', 3),
 ('Tal', 3),
 ('Bien', 4),
 ('supercalifragilisticoespialidoso', 32)]

In [0]:
#Aplica la función argumento sobre los valores de los pares
#El resultado sigue como pares (K,V), en un nuevo RDD
frutas = sc.parallelize([('P','peras,piñas'),('M','manzanas, melones, melocotones'),('N','naranjas,nectarinas'),('L','limones')])
frutas_MAY = frutas.mapValues(lambda elemento: elemento.upper())# mostrar tuplas en mayusculas
print('EN MAYUSCULAS',frutas_MAY.collect())

frutas_min = frutas.mapValues(lambda elemento: elemento.lower())# mostrar tuplas en minusculas
print('en minusculas',frutas_min.collect())

EN MAYUSCULAS [('P', 'PERAS,PIÑAS'), ('M', 'MANZANAS, MELONES, MELOCOTONES'), ('N', 'NARANJAS,NECTARINAS'), ('L', 'LIMONES')]
en minusculas [('P', 'peras,piñas'), ('M', 'manzanas, melones, melocotones'), ('N', 'naranjas,nectarinas'), ('L', 'limones')]


In [0]:
#Agrega todos los elementos del RDD hasta obtener un único valor por clave
#El resultado sigue como pares (K,V) en un nuevo RDD
# la función que se pasa, como en reduce, debe:
#     recibir dos argumentos y devolver un tipo compatible
#     ser conmutativa y asociativa para q se pueda calcular en paralelo
#     a la función se le van a pasar dos valores de elementos con la misma clave
# el tamaño del RDD de salida es igual al numero de claves distintas en el RDD original
r = sc.parallelize([('A',1),('C',4),('A',1),('B',1),('B',4)])
rr = r.reduceByKey(lambda v1,v2 : v1+v2)
print(rr.collect())

[('B', 5), ('C', 4), ('A', 2)]


In [0]:
# que pasaría si ponemos(lambda v1,v2 : str(v1+v2) ?? NPI da fallo
r1 = sc.parallelize([('A',1),('C',4),('A',1),('B',1),('B',4)])
rr1 = r1.reduceByKey(lambda v1,v2 : str(v1+v2)
print(rr1.collect())

  File "<command-1606707781395622>", line 4
    print(rr1.collect())
    ^
SyntaxError: invalid syntax


In [0]:
#Agrupa todos los elementos del RDD para obtener un unico valor por clave con valor igual a la secuencia de valores
# necesário utilizar mapValues(list) para ver el resultado ( se suele utilizar como paso intermedio en un proceso)
# si lo que queremos una ve agrupados los datos es agregarlos (resultado fina) reduceByKey()
r = sc.parallelize([('A',1),('C',2),('A',3),('B',4),('B',5)])
print(r.groupByKey().mapValues(list).collect())
print(r.groupByKey().mapValues(len).collect())


[('B', [4, 5]), ('C', [2]), ('A', [1, 3])]
[('B', 2), ('C', 1), ('A', 2)]


In [0]:
# ordena un RDD por clave de pares (K,V)
#pasandole False ordena de forma inversa
rdd = sc.parallelize([('A',1),('B',6),('C',3),('A',4),('A',5),('B',2)])
res = rdd.sortByKey()# ordena por orden alfabético
print('por orden alafbético:',res.collect())
res = rdd.sortByKey(False)# ordena por orden inverso 
print('por orden inverso:',res.collect())


por orden alafbético: [('A', 1), ('A', 4), ('A', 5), ('B', 6), ('B', 2), ('C', 3)]
por orden inverso: [('C', 3), ('B', 6), ('B', 2), ('A', 1), ('A', 4), ('A', 5)]


In [0]:
#Une dos RDDs (K,V)y(K,W) por clave para dar un RDD (K,(V,W))
#característica especial: devuelve en caso de correspondencia (clave en ambos RDDs) el producto cartesiano, todos con todos de distinto RDD

rdd1 = sc.parallelize([('A',1),('B',2)])
rdd2 = sc.parallelize([('A',4),('B',5),('A',7)])
rddjoin = rdd1.join(rdd2)
print ('resultado join',rddjoin.collect())


resultado join [('B', (2, 5)), ('A', (1, 4)), ('A', (1, 7))]


In [0]:
#semejanza con join() de SQL
#variantes leftOuterJoin() rightOuterJoin() fullOuterJoin()
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])
rddjoin = rdd1.join(rdd2)
print ('join          ',rddjoin.collect())

rddjoin = rdd1.leftOuterJoin(rdd2)
print ('leftOuterJoin ',rddjoin.collect())

rddjoin = rdd1.rightOuterJoin(rdd2)
print ('rightOuterJoin',rddjoin.collect())

rddjoin = rdd1.fullOuterJoin(rdd2)
print ('fullOuterJoin ',rddjoin.collect())

join           [('B', (2, 6)), ('A', (1, 4)), ('A', (1, 5))]
leftOuterJoin  [('B', (2, 6)), ('C', (3, None)), ('A', (1, 4)), ('A', (1, 5))]
rightOuterJoin [('B', (2, 6)), ('D', (None, 7)), ('A', (1, 4)), ('A', (1, 5))]
fullOuterJoin  [('B', (2, 6)), ('D', (None, 7)), ('C', (3, None)), ('A', (1, 4)), ('A', (1, 5))]


In [0]:
#Agrupar las ventas del periodo actual
#Comparar con ventas del periodo anterior

ventas_periodo_actual = sc.parallelize([('Nike',51805),('Puma',42329),('Adidas',63542), ('Puma',27923),('Nike',75335),('Puma',45102),('Adidas',49583), ('Puma',37869),('Adidas',54201), ('Puma',31582),('Nike',62747)])
por_marca_periodo_anterior = sc.parallelize([('Nike',224589),('Puma',166524),('Adidas',219123)])

# agrupacion de ventas por marca periodo actual
por_marca_periodo_actual = ventas_periodo_actual.reduceByKey(lambda v1,v2: v1+v2)
print ('ventas periodo actual  ', por_marca_periodo_actual.collect())

#ventas periodo anterior
print('ventas periodo anterior', por_marca_periodo_anterior.collect())

#unión ventas perriodo anterior y periodo actual por marca
unir_ventas = por_marca_periodo_actual.union(por_marca_periodo_anterior)
#print('unir ventas',unir_ventas.collect())

#diferencia ventas periodo anterior Vs periodo actual
diferencia_ventas = unir_ventas.reduceByKey(lambda v1,v2: v1-v2)
print('diferencia ventas      ',diferencia_ventas.collect())



ventas periodo actual   [('Nike', 189887), ('Puma', 184805), ('Adidas', 167326)]
ventas periodo anterior [('Nike', 224589), ('Puma', 166524), ('Adidas', 219123)]
diferencia ventas       [('Nike', -34702), ('Puma', 18281), ('Adidas', -51797)]


In [0]:

ventas_periodo_actual = sc.parallelize([('Nike',51805),('Puma',42329),('Novedad',18536),
('Adidas',63542),('Puma',27923),('Nike',75335),('Puma',45102),
('Adidas',49583), ('Puma',37869),('Novedad',27196), ('Adidas',54201), ('Puma',31582),('Nike',62747),('Novedad',25409)])
por_marca_periodo_anterior = sc.parallelize([('Nike',224589),('Puma',166524),('Adidas',219123)])

# agrupacion de ventas por marca periodo actual
por_marca_periodo_actual = ventas_periodo_actual.reduceByKey(lambda v1,v2: v1+v2)
print ('ventas periodo actual  ', por_marca_periodo_actual.collect())

#ventas periodo anterior
print('ventas periodo anterior', por_marca_periodo_anterior.collect())

#unión ventas perriodo anterior y periodo actual por marca
unir_ventas = por_marca_periodo_actual.union(por_marca_periodo_anterior)
#print('unir ventas',unir_ventas.collect())

#diferencia ventas periodo anterior Vs periodo actual
diferencia_ventas = unir_ventas.reduceByKey(lambda v1,v2: v1-v2)
print('diferencia ventas      ',diferencia_ventas.collect())

ventas periodo actual   [('Nike', 189887), ('Puma', 184805), ('Adidas', 167326), ('Novedad', 71141)]
ventas periodo anterior [('Nike', 224589), ('Puma', 166524), ('Adidas', 219123)]
diferencia ventas       [('Nike', -34702), ('Novedad', 71141), ('Puma', 18281), ('Adidas', -51797)]
